<a href="https://colab.research.google.com/github/that1guy15/ai_notebooks/blob/main/YOLOv8_ASL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics datasets huggingface_hub opencv-python vidgear[core]

In [ ]:
import torch
from ultralytics import YOLO
from datetime import datetime

# Check if a GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Device: {device}")

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from google.colab import drive

drive.mount('/content/drive')
!ln -s /content/drive/My\ Drive/AI/datasets/sign_language /content/datasets

DS_PATH = "/content/datasets"


# Select a Training Dataset

other datasets for notes:

### Alphabet
asl_alphabet:
  https://www.kaggle.com/datasets/grassknoted/asl-alphabet

### Words
sign_recognition.v2i.yolov8:
  https://universe.roboflow.com/ss-hwnzd/sign_recognition/dataset/2


### Mixed
asllrp:
  http://dai.cs.rutgers.edu/dai/s/index;jsessionid=B88576310F7A2C5EBCF75E98A2C5CB92?redirect=wlasl


In [ ]:
# ASL Datasets:
ASL_seg = f"{DS_PATH}/ASL-SEG.v2i.coco-segmentation"                      #https://universe.roboflow.com/manu-fd9da/asl-seg/dataset/2
ASL_detect = f"{DS_PATH}/ASL-Detection.v1i.yolov8"                        #https://universe.roboflow.com/manu-fd9da/asl-seg/dataset/2
ASL_alphabet = f"{DS_PATH}/american_sign_language_letters.v1-v1.yolov8"   #https://public.roboflow.com/object-detection/american-sign-language-letters

# Train the Model

In [ ]:
# Load the Model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n-seg.pt")  # load a pretrained segmentation model


# Train the model
model.train(data=f"{roboflow}/data.yaml",
            project=f"{roboflow}/training_jobs/result_dir",
            name=f"roboflow_trained_100_epochs_{datetime.utcnow():%Y%m%d_%H%M%S}",
            batch = 80,
            epochs=100)

# Evaluate model performance on the validation set
results = model.val()

# Predict with Your New Model

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image

asl_model = YOLO("/content/datasets/sign_recognition.v2i.yolov8try_again/result_dir/roboflow_trained_100_epochs_20240217_164244/weights/best.pt")

test_dir = '/content/datasets/sign_recognition.v2i.yolov8/valid/images'
test_image = '/W-7-_jpg.rf.1eb713c5c14ed8c1434a5ead73b44290.jpg'

res0 = asl_model.predict(f"{test_dir}/{test_image}", conf = 0.6)[0]

new_result_array = res0.plot()
plt.figure(figsize=(12, 12))
plt.imshow(new_result_array)

In [ ]:
import os
import glob
from pprint import pprint

def find_files(directory, extension):
    if not extension.startswith('.'):
        extension = f".{extension}"

    pattern = os.path.join(directory, '*' + extension)
    return glob.glob(pattern)


In [ ]:
# Run batched inference on a list of images
results = asl_model(test_dir, stream=True)                        # return a generator of Results objects from given directory
#results = asl_model(find_files(test_dir, '.jpg'), stream=True)    # return a generator of Results objects from list of files

# Process results generator
for result in results:
    boxes = result.boxes                # Boxes object for bounding box outputs
    masks = result.masks                # Masks object for segmentation masks outputs
    keypoints = result.keypoints        # Keypoints object for pose outputs
    probs = result.probs                # Probs object for classification outputs
    result.show()                       # display to screen
    #result.save(filename='result.jpg')  # save to disk

    new_result_array = result.plot()
    plt.figure(figsize=(12, 12))
    plt.imshow(new_result_array)

# Predicting from a Streaming Loop or Video

In [ ]:
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow


def cv2_imshow(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    # cv2 stores colors as BGR; convert to RGB
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)

    return plt.imshow(a, **kwargs)

In [ ]:
from ultralytics import YOLO
from vidgear.gears import CamGear

stream = CamGear(
    source="https://www.youtube.com/watch?v=tkMg8g8vVUo",
    stream_mode=True,
    logging=True
    ).start()


while True:
  # read frames
  frame = stream.read()

  # check if frame is None
  if frame is None:
      #if True break the infinite loop
      break

  results = asl_model(frame)
  annotated_frame = results[0].plot()

  cv2.imshow("YOLOv8 Prediction", annotated_frame)
  # Show output window

  key = cv2.waitKey(1) & 0xFF
  # check for 'q' key-press
  if key == ord("q"):
      #if 'q' key-pressed break out
      break

cv2.destroyAllWindows()
# close output window

# safely close video stream.
stream.stop()

In [ ]:
load_ext tensorboard
tensorboard --logdir ultralytics/runs  # replace with 'runs' directory

In [ ]:
results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image

In [ ]:
print(results)